In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [2]:
clean = lambda string: string.strip('\n').strip()

In [3]:
def soupGenerator(url, parser="html.parser", driver_path=r"C:\Apps\chromedriver_win32\chromedriver.exe"):
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    content = driver.page_source
    driver.close()
    return BeautifulSoup(content, parser)

In [4]:
def fillArray(array, soup):
    for a in soup.findAll('tr', attrs={'class':'data-result results'}):
        rank = a.find('td', attrs={'class': 'rank actual-rank'}).text
        country = a.find('td', attrs={'class': 'country'}).text
        speed = a.find('td', attrs={'class': 'speed'}).text
        array.append([int(clean(rank)), clean(country), float(clean(speed))])

In [5]:
time = datetime.now().strftime("%Y %b %d %H:%M:%S")
url = r"https://www.speedtest.net/global-index"
soup = soupGenerator(url)
fixed = soup.find('div', attrs={"id": "column-fixed"})
mobile = soup.find('div', attrs={"id": "column-mobile"})

In [6]:
fixed_rankings = []
fixed_sheet_name = 'Fixed Connection index'
mobile_rankings = []
mobile_sheet_name = 'Mobile Connection index'

In [7]:
for array, soup in [[fixed_rankings, fixed],[mobile_rankings, mobile]]:
    fillArray(array, soup)    

In [8]:
with pd.ExcelWriter('speed_test.xlsx') as writer:
    df1 = pd.DataFrame(data=fixed_rankings, columns=['Rank','Country', 'Speed Mb/s'])
    df1.to_excel(writer, sheet_name=fixed_sheet_name, index=False, startrow=2, startcol=0)
    df2 = pd.DataFrame(data=mobile_rankings, columns=['Rank','Country', 'Speed Mb/s'])
    df2.to_excel(writer, sheet_name=mobile_sheet_name, index=False, startrow=2, startcol=0)
    
    ws = writer.sheets[fixed_sheet_name]
    ws.cell(row=1, column=1).value = 'Updated on: ' + time
    ws2 = writer.sheets[mobile_sheet_name]
    ws2.cell(row=1, column=1).value = 'Updated on: ' + time